In [1]:
import sys
import scipy.io
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler

import deepxde as dde
from deepxde.backend import tf

2024-06-03 21:40:50.221435: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-06-03 21:40:50.257787: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-03 21:40:50.257816: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-03 21:40:50.259041: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-06-03 21:40:50.265394: I tensorflow/core/platform/cpu_feature_guar

Instructions for updating:
non-resource variables are not supported in the long term


Enable just-in-time compilation with XLA.



In [10]:

def get_data(filename, n_train, n_test):
    data = scipy.io.loadmat(filename)
    x = data["x_coor"].astype(np.float32)
    y = data["y_coor"].astype(np.float32)
    t = data["t"].astype(np.float32)
    Vx = data["Vx"].astype(np.float32)
    Vy = data["Vy"].astype(np.float32)
    
    u = np.stack((Vx, Vy), axis=-1)  # Shape: (N, k, n_points, 2)
    u_final = u[:, -1, :, :]  # Shape: (N, n_points, 2)

    u_flattened = u.reshape(u.shape[0], u.shape[1], -1) #Input branch only recieves the 1D array
    # Branch: initial velocity field
    branch = u_flattened[:, 0, :]  # Shape: (N, n_points, 2)
    # Trunk: spatial coordinates grid
    xy = np.vstack((np.ravel(x), np.ravel(y))).T  # N x 2 (x, y)：grid points
    
    # Output: velocity field at the final time step


    # Split into training and testing data
    branch_train = branch[:n_train]
    xy_train = xy
    u_final_train = u_final[:n_train]

    branch_test = branch[-n_test:]
    xy_test = xy
    u_final_test = u_final[-n_test:]

    return (branch_train, xy_train), u_final_train, (branch_test, xy_train), u_final_test

def main():
    n_train = 100  # Adjust based on your dataset size
    n_test = 60  # Adjust based on your dataset size
    nx = 6400

    data_path = "/home/liruixiang/data/gepup/train_data_new.mat"
    x_train, u_final_train, x_test, u_final_test = get_data(data_path, n_train, n_test)
    
    # Split u_final_train and u_final_test into Vx and Vy parts
    u_final_train_vx = u_final_train[:, :, 0]
    u_final_train_vy = u_final_train[:, :, 1]
    u_final_test_vx = u_final_test[:, :, 0]
    u_final_test_vy = u_final_test[:, :, 1]
    
    # train_x = [branch_train, xy_train]
    # test_x = [branch_test, xy_test]

    # Create datasets for Vx and Vy
    data_vx = dde.data.Triple(
        x_train, u_final_train_vx,
        x_test, u_final_test_vx
    )
    data_vy = dde.data.Triple(
        x_train, u_final_train_vy,
        x_test, u_final_test_vy
    )

    # 创建两个子网络，分别生成Vx和Vy
    net_vx = dde.maps.DeepONetCartesianProd(
        [nx * 2, 128, 128], [2, 128, 128], "relu", "Glorot normal"
    )

    net_vy = dde.maps.DeepONetCartesianProd(
        [nx * 2, 128, 128], [2, 128, 128], "relu", "Glorot normal"
    )

    # 创建模型
    model_vx = dde.Model(data_vx, net_vx)
    model_vy = dde.Model(data_vy, net_vy)

    # 编译模型
    model_vx.compile(
        "adam",
        lr=1e-3,
        decay=("inverse time", 1, 1e-4),
        metrics=["mean l2 relative error"],
    )
    model_vy.compile(
        "adam",
        lr=1e-3,
        decay=("inverse time", 1, 1e-4),
        metrics=["mean l2 relative error"],
    )

    # 训练模型
    losshistory_vx, train_state_vx = model_vx.train(epochs=250000, batch_size=None)
    losshistory_vy, train_state_vy = model_vy.train(epochs=250000, batch_size=None)

    # 预测
    y_pred_vx = model_vx.predict(data_vx.test_x)
    y_pred_vy = model_vy.predict(data_vy.test_x)

    # 保存预测结果
    np.savetxt("y_pred_vx_deeponet.dat", y_pred_vx[0].reshape(nx, 1))
    np.savetxt("y_pred_vy_deeponet.dat", y_pred_vy[0].reshape(nx, 1))
    np.savetxt("y_true_vx_deeponet.dat", data_vx.test_y[0].reshape(nx, 1))
    np.savetxt("y_true_vy_deeponet.dat", data_vy.test_y[0].reshape(nx, 1))
    np.savetxt("y_error_vx_deeponet.dat", (y_pred_vx[0] - data_vx.test_y[0]).reshape(nx, 1))
    np.savetxt("y_error_vy_deeponet.dat", (y_pred_vy[0] - data_vy.test_y[0]).reshape(nx, 1))

if __name__ == "__main__":
    # Enable memory growth for GPUs
    physical_devices = tf.config.experimental.list_physical_devices('GPU')
    if physical_devices:
        try:
            for device in physical_devices:
                tf.config.experimental.set_memory_growth(device, True)
        except RuntimeError as e:
            print(e)
    
    main()


Compiling model...
Building DeepONetCartesianProd...
'build' took 0.039181 s



/home/liruixiang/.conda/envs/deeponet/lib/python3.9/site-packages/deepxde/nn/tensorflow_compat_v1/deeponet.py:549: UserWarning: `tf.layers.dense` is deprecated and will be removed in a future version. Please use `tf.keras.layers.Dense` instead.
  y_func = tf.layers.dense(
/home/liruixiang/.conda/envs/deeponet/lib/python3.9/site-packages/deepxde/nn/tensorflow_compat_v1/deeponet.py:556: UserWarning: `tf.layers.dense` is deprecated and will be removed in a future version. Please use `tf.keras.layers.Dense` instead.
  y_func = tf.layers.dense(
/home/liruixiang/.conda/envs/deeponet/lib/python3.9/site-packages/deepxde/nn/tensorflow_compat_v1/deeponet.py:570: UserWarning: `tf.layers.dense` is deprecated and will be removed in a future version. Please use `tf.keras.layers.Dense` instead.
  y_loc = tf.layers.dense(
2024-06-03 19:33:31.691559: W tensorflow/core/common_runtime/gpu/gpu_bfc_allocator.cc:47] Overriding orig_value setting because the TF_FORCE_GPU_ALLOW_GROWTH environment variable is 

'compile' took 0.258510 s

Compiling model...
Building DeepONetCartesianProd...
'build' took 0.034974 s



2024-06-03 19:33:31.841964: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1929] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 22456 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3090, pci bus id: 0000:02:00.0, compute capability: 8.6


'compile' took 0.287632 s

Training model...



2024-06-03 19:33:32.098160: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:388] MLIR V1 optimization pass is not enabled
2024-06-03 19:33:32.241954: I external/local_xla/xla/service/service.cc:168] XLA service 0x7fdfac009230 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-06-03 19:33:32.241993: I external/local_xla/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce RTX 3090, Compute Capability 8.6
2024-06-03 19:33:32.250016: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-06-03 19:33:32.322799: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8902
I0000 00:00:1717414412.700194  868136 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


Step      Train loss    Test loss     Test metric   
0         [1.03e-02]    [9.85e-03]    [1.27e+00]    
1000      [1.30e-04]    [1.29e-04]    [1.72e-01]    
2000      [6.67e-05]    [6.83e-05]    [1.35e-01]    
3000      [4.93e-05]    [5.17e-05]    [1.22e-01]    
4000      [4.37e-05]    [4.61e-05]    [1.16e-01]    
5000      [4.12e-05]    [4.35e-05]    [1.13e-01]    
6000      [3.95e-05]    [4.17e-05]    [1.11e-01]    
7000      [6.53e-05]    [6.59e-05]    [1.31e-01]    
8000      [3.54e-05]    [3.73e-05]    [1.10e-01]    
9000      [2.89e-05]    [3.08e-05]    [1.09e-01]    
10000     [2.73e-05]    [2.90e-05]    [1.10e-01]    
11000     [3.04e-05]    [3.16e-05]    [1.13e-01]    
12000     [8.70e-05]    [8.63e-05]    [1.47e-01]    
13000     [2.63e-05]    [2.71e-05]    [1.01e-01]    
14000     [2.88e-05]    [2.92e-05]    [9.88e-02]    
15000     [1.88e-05]    [1.94e-05]    [8.07e-02]    
16000     [1.65e-05]    [1.70e-05]    [7.43e-02]    
17000     [1.82e-05]    [1.85e-05]    [7.20e-0

2024-06-03 20:08:58.483354: W tensorflow/c/c_api.cc:305] Operation '{name:'Variable_2/Adam_1/Assign' id:659 op device:{requested: '', assigned: ''} def:{{{node Variable_2/Adam_1/Assign}} = Assign[T=DT_FLOAT, _class=["loc:@Variable_2"], _has_manual_control_dependencies=true, use_locking=true, validate_shape=true](Variable_2/Adam_1, Variable_2/Adam_1/Initializer/zeros)}}' was changed by setting attribute after it was run by a session. This mutation will have no effect, and will trigger an error in the future. Either don't modify nodes after running them or create a new session.
